# Fiddler LangGraph SDK: Custom Instrumentation with Decorators and Context Managers

This notebook demonstrates the **decorator-based** and **manual** instrumentation modes introduced in Fiddler LangGraph SDK v1.4.0. These modes give you fine-grained control over trace structure and metadata — ideal for custom application logic, non-LangGraph components, and advanced observability patterns.

## What You'll Learn

1. **Decorator basics** — instrument any function with `@trace()`
2. **Span metadata** — use `get_current_span()` to add LLM-specific attributes
3. **Span types** — `generation`, `tool`, `chain` with type-specific helper methods
4. **Manual instrumentation** — `start_as_current_span()` and `start_span()` context managers
5. **Combining approaches** — mix decorators and manual spans in one trace
6. **Async support** — `@trace()` works with async functions automatically

## When to Use Custom Instrumentation

| Approach | Best for |
|----------|----------|
| **Auto-instrumentation** (`LangGraphInstrumentor`) | LangGraph agents — zero-code setup |
| **Decorator** (`@trace()`) | Custom Python functions, non-framework code, quick setup |
| **Manual** (`start_as_current_span()`) | Loops, conditional logic, dynamic span names, maximum control |

## Prerequisites

- Python 3.10+
- OpenAI API key
- Fiddler instance with API key and application ID

In [ ]:
%pip install fiddler-langgraph openai python-dotenv

## Environment Setup

Create a `.env` file in your working directory with your credentials:

```text
FIDDLER_URL=https://your-instance.fiddler.ai
FIDDLER_APPLICATION_ID=your-genai-application-id
FIDDLER_API_KEY=your-fiddler-access-token
OPENAI_API_KEY=your-openai-api-key
```

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

required_vars = ["FIDDLER_API_KEY", "FIDDLER_APPLICATION_ID", "FIDDLER_URL", "OPENAI_API_KEY"]
missing = [v for v in required_vars if not os.getenv(v)]

if missing:
    raise EnvironmentError(f"Missing environment variables: {missing}")

print(f"Fiddler URL: {os.getenv('FIDDLER_URL')}")
print(f"Application ID: {os.getenv('FIDDLER_APPLICATION_ID')}")
print("All credentials configured.")

## Initialize the Fiddler Client

Creating a `FiddlerClient` automatically registers it as the **global singleton**. The `@trace()` decorator and `get_client()` use this singleton by default, so you only need to create the client once.

In [ ]:
from fiddler_langgraph import FiddlerClient

fdl_client = FiddlerClient(
    api_key=os.getenv("FIDDLER_API_KEY"),
    application_id=os.getenv("FIDDLER_APPLICATION_ID"),
    url=os.getenv("FIDDLER_URL"),
    console_tracer=True,  # Print spans to console for debugging
)

print(f"FiddlerClient initialized. Console tracing enabled.")

In [ ]:
# Initialize the OpenAI client for LLM calls throughout this notebook
from openai import OpenAI

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

---

## Part 1: Basic `@trace()` Decorator

The `@trace()` decorator instruments any Python function — sync or async — with a single line. By default it captures the function's arguments as input and return value as output.

In [ ]:
from fiddler_langgraph import trace


@trace()
def greet(name: str) -> str:
    """A simple function to demonstrate @trace() basics."""
    return f"Hello, {name}! Welcome to Fiddler."


# Call the function — a span is created automatically
result = greet("Alice")
print(result)

# The span captures:
#   - name: "greet" (from the function name)
#   - input: {"name": "Alice"}
#   - output: "Hello, Alice! Welcome to Fiddler."

### Controlling Input/Output Capture

Use `capture_input` and `capture_output` to control what gets recorded. This is useful when arguments contain sensitive data or when the return value is large.

In [ ]:
@trace(capture_input=False, capture_output=False)
def process_sensitive_data(user_email: str, ssn: str) -> dict:
    """Process sensitive data without capturing PII in traces."""
    return {"status": "processed", "user": user_email}


result = process_sensitive_data("alice@example.com", "123-45-6789")
print(result)
# The span is created but input/output are not recorded

### Custom Span Names

By default, the span name is the function name. Use the `name` parameter to override it.

In [ ]:
@trace(name="data_validation")
def validate(data: dict) -> bool:
    """Validate input data. Span will be named 'data_validation', not 'validate'."""
    return "query" in data and len(data["query"]) > 0


is_valid = validate({"query": "What is the weather in Tokyo?"})
print(f"Valid: {is_valid}")

---

## Part 2: Adding Metadata with `get_current_span()`

Inside a `@trace()`-decorated function, call `get_current_span()` to access the active span and set additional attributes. This is how you add LLM-specific metadata like model name, token usage, and custom attributes.

In [ ]:
from fiddler_langgraph import get_current_span


@trace(as_type="generation", capture_input=False, capture_output=False)
def ask_llm(question: str) -> str:
    """Call OpenAI and capture LLM metadata via get_current_span()."""
    span = get_current_span(as_type="generation")

    # Set LLM metadata before the call
    if span:
        span.set_model("gpt-4o-mini")
        span.set_system("openai")
        span.set_user_prompt(question)

    # Make the OpenAI API call
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}],
        temperature=0.3,
    )

    answer = response.choices[0].message.content

    # Set output metadata after the call
    if span:
        span.set_completion(answer)
        if response.usage:
            span.set_usage(
                input_tokens=response.usage.prompt_tokens,
                output_tokens=response.usage.completion_tokens,
                total_tokens=response.usage.total_tokens,
            )

    return answer


answer = ask_llm("What is the capital of France?")
print(f"Answer: {answer}")

### Custom Attributes

Use `set_attribute()` to attach any custom key-value pair to a span. Custom attributes appear as `fiddler.span.user.<key>` in Fiddler and can be used for filtering and analysis.

In [ ]:
@trace(as_type="generation", capture_input=False, capture_output=False)
def ask_llm_with_context(question: str, department: str) -> str:
    """LLM call with custom business attributes."""
    span = get_current_span(as_type="generation")

    if span:
        span.set_model("gpt-4o-mini")
        span.set_system("openai")
        span.set_user_prompt(question)
        # Custom attributes for business context
        span.set_attribute("department", department)
        span.set_attribute("priority", "high")

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}],
        temperature=0.3,
    )

    answer = response.choices[0].message.content

    if span:
        span.set_completion(answer)
        if response.usage:
            span.set_usage(
                input_tokens=response.usage.prompt_tokens,
                output_tokens=response.usage.completion_tokens,
            )

    return answer


answer = ask_llm_with_context("Summarize the benefits of AI observability.", department="engineering")
print(f"Answer: {answer[:200]}...")

---

## Part 3: Span Types and Helper Methods

The `as_type` parameter determines the span's semantic type and which helper methods are available:

| Span Type | Wrapper Class | Purpose | Key Helpers |
|-----------|--------------|---------|-------------|
| `generation` | `FiddlerGeneration` | LLM calls | `set_model()`, `set_user_prompt()`, `set_completion()`, `set_usage()` |
| `tool` | `FiddlerTool` | Tool/function executions | `set_tool_name()`, `set_tool_input()`, `set_tool_output()` |
| `chain` | `FiddlerChain` | Workflow orchestration | `set_input()`, `set_output()` |
| `span` | `FiddlerSpan` | General-purpose (default) | `set_input()`, `set_output()`, `set_attribute()` |

All types share common helpers: `set_agent_name()`, `set_conversation_id()`, `set_attribute()`.

### Tool Spans

Use `as_type='tool'` for functions that perform tool-like operations (API calls, database queries, searches).

In [ ]:
# Simulated knowledge base
HOTEL_KNOWLEDGE = {
    "Tokyo": "Mandarin Oriental Tokyo — luxury hotel in Nihonbashi with views of Mount Fuji.",
    "Paris": "The Ritz Paris — historic hotel on Place Vendome with Michelin-starred dining.",
    "London": "The Savoy — legendary hotel on the Strand, famous for afternoon tea.",
}


@trace(as_type="tool", name="search_hotel")
def search_hotel(city: str) -> str:
    """Search the knowledge base for hotel information."""
    span = get_current_span(as_type="tool")
    if span:
        span.set_tool_name("search_hotel")
        span.set_tool_input({"city": city})

    result = HOTEL_KNOWLEDGE.get(city, f"No hotel information found for {city}.")

    if span:
        span.set_tool_output(result)

    return result


hotel_info = search_hotel("Tokyo")
print(f"Hotel info: {hotel_info}")

### Chain Spans with Nesting

Use `as_type='chain'` for orchestration functions. When decorated functions call each other, the `@trace()` decorator automatically establishes parent-child relationships.

In [ ]:
import json


@trace(as_type="generation", capture_input=False, capture_output=False, model="gpt-4o-mini", system="openai")
def recommend_hotel(city: str, context: str) -> str:
    """Use LLM to generate a hotel recommendation based on search results."""
    span = get_current_span(as_type="generation")

    prompt = f"Based on this hotel info: {context}\nWrite a one-sentence recommendation for a traveler visiting {city}."

    if span:
        span.set_user_prompt(prompt)

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
    )

    answer = response.choices[0].message.content

    if span:
        span.set_completion(answer)
        if response.usage:
            span.set_usage(
                input_tokens=response.usage.prompt_tokens,
                output_tokens=response.usage.completion_tokens,
            )

    return answer


@trace(as_type="chain", name="travel_advisor")
def travel_advisor(city: str) -> dict:
    """Orchestrate a hotel search and LLM recommendation.

    This creates a trace with three spans:
      travel_advisor (chain)
        -> search_hotel (tool)
        -> recommend_hotel (generation)
    """
    span = get_current_span(as_type="chain")
    if span:
        span.set_agent_name("travel_advisor")
        span.set_input({"city": city})

    # Step 1: Search for hotel info (tool span)
    hotel_info = search_hotel(city)

    # Step 2: Generate recommendation (generation span)
    recommendation = recommend_hotel(city, hotel_info)

    result = {"city": city, "hotel_info": hotel_info, "recommendation": recommendation}

    if span:
        span.set_output(result)

    return result


# Run the chain — observe the parent-child span hierarchy
advice = travel_advisor("Paris")
print(f"City: {advice['city']}")
print(f"Hotel: {advice['hotel_info']}")
print(f"Recommendation: {advice['recommendation']}")

---

## Part 4: Manual Instrumentation with Context Managers

Manual instrumentation gives you full control over span boundaries. Use it when you need to:
- Instrument loops where each iteration should be a separate span
- Create spans with dynamic names
- Control exactly when a span starts and ends

### `start_as_current_span()` — Recommended

The context manager automatically ends the span and records exceptions.

In [ ]:
import uuid


def multi_city_search(cities: list[str]) -> dict:
    """Search hotels in multiple cities using manual instrumentation.

    Each city gets its own tool span, all nested under one chain span.
    This pattern is ideal for loops where each iteration should be tracked.
    """
    session_id = str(uuid.uuid4())
    results = {}

    with fdl_client.start_as_current_span("multi_city_search", as_type="chain") as chain:
        chain.set_agent_name("hotel_search")
        chain.set_conversation_id(session_id)
        chain.set_input({"cities": cities})

        for city in cities:
            # Each iteration creates a child tool span
            with fdl_client.start_as_current_span(f"search_{city.lower()}", as_type="tool") as tool:
                tool.set_tool_name("search_hotel")
                tool.set_tool_input({"city": city})

                info = HOTEL_KNOWLEDGE.get(city, f"No information for {city}.")
                results[city] = info

                tool.set_tool_output(info)

        chain.set_output(results)

    return results


results = multi_city_search(["Tokyo", "Paris", "London"])
for city, info in results.items():
    print(f"{city}: {info}")

### `start_span()` — Explicit Control

Use `start_span()` when you need to end the span at a specific point rather than at the end of a `with` block. You **must** call `span.end()` manually.

In [ ]:
def conditional_search(city: str, include_recommendation: bool) -> dict:
    """Demonstrate start_span() for explicit lifecycle control."""
    # Start span manually
    span = fdl_client.start_span("conditional_search", as_type="chain")
    span.set_input({"city": city, "include_recommendation": include_recommendation})

    result = {"city": city}

    try:
        # Always search for hotel info
        with fdl_client.start_as_current_span("hotel_lookup", as_type="tool") as tool:
            tool.set_tool_name("search_hotel")
            tool.set_tool_input({"city": city})
            info = HOTEL_KNOWLEDGE.get(city, f"No information for {city}.")
            result["hotel_info"] = info
            tool.set_tool_output(info)

        # Conditionally add an LLM recommendation
        if include_recommendation:
            with fdl_client.start_as_current_span("generate_recommendation", as_type="generation") as gen:
                gen.set_model("gpt-4o-mini")
                gen.set_system("openai")
                prompt = f"In one sentence, why should someone visit {city}?"
                gen.set_user_prompt(prompt)

                response = openai_client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.5,
                )
                recommendation = response.choices[0].message.content
                result["recommendation"] = recommendation
                gen.set_completion(recommendation)

        span.set_output(result)
    finally:
        # Always end the span
        span.end()

    return result


# With recommendation
result = conditional_search("London", include_recommendation=True)
print(f"City: {result['city']}")
print(f"Hotel: {result['hotel_info']}")
print(f"Recommendation: {result.get('recommendation', 'N/A')}")

---

## Part 5: Combining Decorators and Manual Spans

You can mix decorator and manual instrumentation in the same trace. Decorated functions automatically become children of any active manual span, and manual spans inside decorated functions become children of the decorator's span.

This example builds a simple agent loop that:
1. Uses `@trace()` for the LLM call and tool functions
2. Uses manual spans for the orchestration loop (where iteration count is dynamic)

In [ ]:
# Tool functions using decorators
@trace(as_type="tool", name="book_flight")
def book_flight(origin: str, destination: str) -> str:
    """Book a flight between two cities."""
    span = get_current_span(as_type="tool")
    if span:
        span.set_tool_name("book_flight")
        span.set_tool_input({"origin": origin, "destination": destination})

    result = f"Flight booked: {origin} -> {destination}, confirmation #FL{uuid.uuid4().hex[:6].upper()}"

    if span:
        span.set_tool_output(result)
    return result


# LLM call using decorator
@trace(as_type="generation", capture_input=False, capture_output=False, model="gpt-4o-mini", system="openai")
def plan_trip(user_request: str) -> dict:
    """Ask the LLM to plan a trip and return structured output."""
    span = get_current_span(as_type="generation")

    system_prompt = (
        "You are a travel planner. Given a request, respond with a JSON object "
        "containing: {\"action\": \"book_flight\" or \"search_hotel\" or \"done\", "
        "\"origin\": \"city\", \"destination\": \"city\", \"message\": \"summary\"}. "
        "Respond with ONLY the JSON object, no markdown."
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_request},
    ]

    if span:
        span.set_user_prompt(user_request)
        span.set_messages(messages)

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.0,
    )

    content = response.choices[0].message.content

    if span:
        span.set_completion(content)
        if response.usage:
            span.set_usage(
                input_tokens=response.usage.prompt_tokens,
                output_tokens=response.usage.completion_tokens,
            )

    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {"action": "done", "message": content}


# Orchestration using manual spans (dynamic loop)
def travel_agent(user_request: str) -> str:
    """Simple agent loop mixing decorated functions with manual spans."""
    session_id = str(uuid.uuid4())

    with fdl_client.start_as_current_span("travel_agent", as_type="chain") as root:
        root.set_agent_name("travel_agent")
        root.set_conversation_id(session_id)
        root.set_input({"request": user_request})

        # Agent loop — manual span for each iteration
        max_iterations = 3
        results = []

        for i in range(max_iterations):
            with fdl_client.start_as_current_span(f"iteration_{i}", as_type="chain") as step:
                step.set_attribute("iteration", i)

                # Decorated function called inside manual span — becomes a child
                plan = plan_trip(user_request)
                action = plan.get("action", "done")

                if action == "book_flight":
                    result = book_flight(
                        origin=plan.get("origin", "Unknown"),
                        destination=plan.get("destination", "Unknown"),
                    )
                    results.append(result)
                elif action == "search_hotel":
                    result = search_hotel(plan.get("destination", "Unknown"))
                    results.append(result)
                else:
                    results.append(plan.get("message", "Done."))
                    break

                step.set_output({"action": action, "result": results[-1]})

        final_output = "\n".join(results)
        root.set_output(final_output)

    return final_output


output = travel_agent("I need to fly from Boston to Tokyo")
print(f"Agent output:\n{output}")

---

## Part 6: Async Support

The `@trace()` decorator automatically detects async functions. No special syntax is needed — just decorate your `async def` as usual.

In [ ]:
import asyncio


@trace(as_type="generation", capture_input=False, capture_output=False, model="gpt-4o-mini", system="openai")
async def async_ask_llm(question: str) -> str:
    """Async LLM call — @trace() handles async automatically."""
    span = get_current_span(as_type="generation")
    if span:
        span.set_user_prompt(question)

    # Using synchronous OpenAI client in async context for simplicity.
    # In production, use the AsyncOpenAI client.
    response = await asyncio.to_thread(
        openai_client.chat.completions.create,
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}],
        temperature=0.3,
    )

    answer = response.choices[0].message.content
    if span:
        span.set_completion(answer)
        if response.usage:
            span.set_usage(
                input_tokens=response.usage.prompt_tokens,
                output_tokens=response.usage.completion_tokens,
            )
    return answer


@trace(as_type="chain", name="parallel_questions")
async def ask_multiple(questions: list[str]) -> list[str]:
    """Run multiple LLM calls concurrently — each gets its own generation span."""
    span = get_current_span(as_type="chain")
    if span:
        span.set_input({"questions": questions})

    answers = await asyncio.gather(*[async_ask_llm(q) for q in questions])

    if span:
        span.set_output({"answers": list(answers)})
    return list(answers)


# Run async functions
questions = [
    "What is the tallest building in Tokyo?",
    "What is the most famous museum in Paris?",
    "What is the best time to visit London?",
]
answers = await ask_multiple(questions)

for q, a in zip(questions, answers):
    print(f"Q: {q}")
    print(f"A: {a[:150]}...\n")

---

## Context Isolation and `is_fiddler_span()`

Each `FiddlerClient` operates in its own isolated OpenTelemetry context. This means Fiddler traces never interfere with other OTel-instrumented libraries in your application.

Use `is_fiddler_span()` to verify whether a span belongs to Fiddler's tracer.

In [ ]:
from fiddler_langgraph import get_client
from fiddler_langgraph.core.utils import is_fiddler_span

# Verify the global client singleton
client = get_client()
print(f"Global client retrieved: {client is not None}")
print(f"Same instance as fdl_client: {client is fdl_client}")

# Verify span ownership inside a trace
with fdl_client.start_as_current_span("ownership_check", as_type="span") as span:
    print(f"Is Fiddler span: {is_fiddler_span(span)}")

---

## Cleanup

Flush remaining spans and shut down the client. In production, `FiddlerClient` registers an `atexit` handler that does this automatically, but for notebooks it is best to call `shutdown()` explicitly.

In [ ]:
fdl_client.force_flush(timeout_millis=5000)
fdl_client.shutdown()
print("All spans flushed and client shut down.")

---

## Summary

This notebook covered the custom instrumentation modes introduced in Fiddler LangGraph SDK v1.4.0:

| Concept | What You Learned |
|---------|------------------|
| `@trace()` decorator | Instrument any function with one line; automatic I/O capture |
| `get_current_span()` | Access the active span to set LLM metadata, token usage, and custom attributes |
| Span types | `generation`, `tool`, `chain` with type-specific helper methods |
| `start_as_current_span()` | Manual context manager with automatic lifecycle management |
| `start_span()` | Explicit span control when you need to end spans conditionally |
| Mixing approaches | Decorators and manual spans compose naturally in the same trace |
| Async support | `@trace()` works with async functions automatically |
| Context isolation | Fiddler traces never interfere with other OTel tracers |

## Next Steps

- **Auto-instrumentation**: Use `LangGraphInstrumentor` for zero-code LangGraph agent monitoring — see the [Quick Start Guide](https://docs.fiddler.ai/developers/quick-starts/langgraph-sdk-quick-start)
- **Advanced patterns**: Production configuration, multi-agent systems, and performance optimization — see the [Advanced Guide](https://docs.fiddler.ai/developers/tutorials/llm-monitoring/langgraph-sdk-advanced)
- **SDK API reference**: Full API documentation for all classes and functions — see the [SDK API Reference](https://docs.fiddler.ai/sdk-api/langgraph/)
- **Integration guide**: Comprehensive overview of all three instrumentation approaches — see the [LangGraph SDK Integration](https://docs.fiddler.ai/integrations/agentic-ai/langgraph-sdk)

---

**Support:** [support@fiddler.ai](mailto:support@fiddler.ai) | [Fiddler Documentation](https://docs.fiddler.ai)